In [19]:
import requests
from PIL import Image
from io import BytesIO

# Function to search for books by title and author using Google Books API
def search_books(book_name, author_name):
    # URL for Google Books API
    url = f"https://www.googleapis.com/books/v1/volumes?q=intitle:{book_name}+inauthor:{author_name}"
    
    # Send a GET request to Google Books API
    response = requests.get(url)
    
    if response.status_code == 200:
        books = response.json()
        
        if "items" in books:
            # Loop through the search results and print book details
            for idx, book in enumerate(books['items']):
                title = book['volumeInfo'].get('title', 'No title')
                authors = book['volumeInfo'].get('authors', ['No authors listed'])
                description = book['volumeInfo'].get('description', 'No description available')
                preview_link = book['volumeInfo'].get('previewLink', 'No preview available')

                print(f"Result {idx + 1}:")
                print(f"Title: {title}")
                print(f"Author(s): {', '.join(authors)}")
                print(f"Description: {description}")
                print(f"Preview Link: {preview_link}")

                # Fetch book cover image if available
                if 'imageLinks' in book['volumeInfo']:
                    image_links = book['volumeInfo']['imageLinks']
                    show_book_images(image_links)
                print("\n")
        else:
            print("No books found for the given search criteria.")
    else:
        print("Error: Unable to fetch data from Google Books API")

# Function to display book images from the image links (cover images)
def show_book_images(image_links):
    # Check if images are available
    if 'thumbnail' in image_links:
        thumbnail_url = image_links['thumbnail']
        print(f"Book Thumbnail: {thumbnail_url}")
        # Display the thumbnail image
        display_image_from_url(thumbnail_url)
    elif 'small' in image_links:
        small_url = image_links['small']
        print(f"Book Small Image: {small_url}")
        # Display the small image
        display_image_from_url(small_url)
    elif 'medium' in image_links:
        medium_url = image_links['medium']
        print(f"Book Medium Image: {medium_url}")
        # Display the medium image
        display_image_from_url(medium_url)
    elif 'large' in image_links:
        large_url = image_links['large']
        print(f"Book Large Image: {large_url}")
        # Display the large image
        display_image_from_url(large_url)

# Function to display an image from a URL
def display_image_from_url(image_url):
    try:
        # Fetch the image from the URL
        response = requests.get(image_url)
        img = Image.open(BytesIO(response.content))
        img.show()  # Show the image using the default image viewer
    except Exception as e:
        print(f"Error displaying image: {e}")

# Main function to input book name, author, and API key
if __name__ == "__main__":
    # Input details from the user
    book_name = input("Enter the book_name: ")
    author_name = input("Enter the author name: ")
    
    search_books(book_name, author_name)

Enter the book_name:  From ML Algorithm to GenAI & LLMs
Enter the author name:  Aman kharwal


Result 1:
Title: From ML Algorithms to GenAI & LLMs
Author(s): Aman Kharwal
Description: From ML Algorithms to GenAI & LLMs, Written by Aman Kharwal, founder of Statso.io, is the second edition of the book - Machine Learning Algorithms: Handbook. This book offers a comprehensive and expanded guide through the evolving world of machine learning and generative AI. Whether you are an experienced data scientist or just starting, this edition delivers practical insights and clear explanations of essential concepts like regression, classification, clustering, deep learning, and time series forecasting. This edition introduces two new chapters: "Mastering GenAI and LLMs" and "Understanding GANs for Generative AI with a Hands-on Project", which provide deep dives into large language models and generative adversarial networks (GANs). With hands-on Python code snippets and real-world project examples, the book bridges the gap between theory and application, offering you the tools to apply machin

In [8]:
pip install requests fpdf pdf2image

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40714 sha256=1f6c5773ee6f437619a8a5a0719dfa77f164c4344859872a43f5edd679d529ad
  Stored in directory: c:\users\my dell\appdata\local\pip\cache\wheels\6e\62\11\dc73d78e40a218ad52e7451f30166e94491be013a7850b5d75
Successfully built fpdf
Note: you may need to restart the kernel to use updated packages.


In [21]:
pip install pytesseract pdf2image fpdf

Note: you may need to restart the kernel to use updated packages.


In [37]:
import requests
from fpdf import FPDF
from pdf2image import convert_from_path

# Function to search for books by title and author using Google Books API
def search_books(book_name, author_name):
    # URL for Google Books API
    url = f"https://www.googleapis.com/books/v1/volumes?q=intitle:{book_name}+inauthor:{author_name}"
    
    # Send a GET request to Google Books API
    response = requests.get(url)
    
    if response.status_code == 200:
        books = response.json()
        
        if "items" in books:
            # Loop through the search results and print book details
            for idx, book in enumerate(books['items']):
                title = book['volumeInfo'].get('title', 'No title')
                authors = book['volumeInfo'].get('authors', ['No authors listed'])
                description = book['volumeInfo'].get('description', 'No description available')
                preview_link = book['volumeInfo'].get('previewLink', 'No preview available')
                print(f"Result {idx + 1}:")
                print(f"Title: {title}")
                print(f"Author(s): {', '.join(authors)}")
                print(f"Description: {description}")
                print(f"Preview Link: {preview_link}")
                
                # Check if book has a PDF link
                if 'accessInfo' in book and 'pdf' in book['accessInfo']:
                    pdf_info = book['accessInfo']['pdf']
                    if pdf_info.get('isAvailable', False):
                        book_url = pdf_info.get('acsTokenLink', '')
                        if book_url:
                            print(f"Book available in PDF format: {book_url}")
                            return book_url
                print("\n")
        else:
            print("No books found for the given search criteria.")
    else:
        print("Error: Unable to fetch data from Google Books API")
    return None

# Function to convert the book text into a PDF
def text_to_pdf(book_text, output_pdf_path):
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.add_page()

    pdf.set_font("Arial", size=12)
    
    # Split the text into pages (for simplicity, 1000 characters per page)
    page_size = 1000
    for i in range(0, len(book_text), page_size):
        pdf.multi_cell(0, 10, book_text[i:i + page_size])
        pdf.add_page()

    # Output the PDF
    pdf.output(output_pdf_path)

# Function to convert a PDF to images (one image per page)
def pdf_to_images(pdf_path):
    pages = convert_from_path(pdf_path)
    
    for i, page in enumerate(pages):
        image_path = f"page_{i + 1}.png"
        page.save(image_path, 'PNG')
        print(f"Page {i + 1} saved as {image_path}")

# Main function to convert book to PDF
def main():
    book_name = input("Enter the book name: ")
    author_name = input("Enter the author name: ")

    # Search for the book
    book_url = search_books(book_name, author_name)
    
    if book_url:
        # Download book content (this example assumes public domain content from Project Gutenberg)
        # Note: You may need to implement a proper download function if the URL is valid.
        # For now, we will skip this part as Google Books API may not provide direct content.
        
        # Example placeholder for book text
        book_text = "This is a placeholder for the book content. Replace this with actual content"
def search_books(book_name, author_name):
    url = f"https://www.googleapis.com/books/v1/volumes?q=intitle:{book_name}+inauthor:{author_name}"
    response = requests.get(url)
    
    if response.status_code == 200:
        books = response.json()
        if "items" in books:
            for book in books['items']:
                if 'accessInfo' in book and 'pdf' in book['accessInfo']:
                    pdf_info = book['accessInfo']['pdf']
                    if pdf_info.get('isAvailable', False):
                        return pdf_info.get('acsTokenLink', '')
    return None

# Example usage
book_url = search_books("From ml algorithm to genai & llms", "Aman kharwal")
if book_url:
    print(f"Download link for PDF: {book_url}")
else:
    print("No PDF available for this book.")

Download link for PDF: http://books.google.co.in/books/download/From_ML_Algorithms_to_GenAI_LLMs-sample-pdf.acsm?id=LCAqEQAAQBAJ&format=pdf&output=acs4_fulfillment_token&dl_type=sample&source=gbs_api


In [38]:
import requests

# Function to search for books by title and author using Google Books API
def search_books(book_name, author_name):
    # URL for Google Books API
    url = f"https://www.googleapis.com/books/v1/volumes?q=intitle:{book_name}+inauthor:{author_name}"
    
    # Send a GET request to Google Books API
    response = requests.get(url)
    
    if response.status_code == 200:
        books = response.json()
        
        if "items" in books:
            # Loop through the search results and print book details
            for idx, book in enumerate(books['items']):
                title = book['volumeInfo'].get('title', 'No title')
                authors = book['volumeInfo'].get('authors', ['No authors listed'])
                print(f"Result {idx + 1}:")
                print(f"Title: {title}")
                print(f"Author(s): {', '.join(authors)}")
                
                # Check if book has a PDF link
                if 'accessInfo' in book and 'pdf' in book['accessInfo']:
                    pdf_info = book['accessInfo']['pdf']
                    if pdf_info.get('isAvailable', False):
                        book_url = pdf_info.get('acsTokenLink', '')
                        if book_url:
                            print(f"Book available in PDF format: {book_url}")
                            return book_url
                print("\n")
        else:
            print("No books found for the given search criteria.")
    else:
        print("Error: Unable to fetch data from Google Books API")
    return None

# Function to download the PDF from the URL
def download_pdf(pdf_url, output_filename):
    try:
        response = requests.get(pdf_url)
        response.raise_for_status()  # Raise an error for bad responses
        with open(output_filename, 'wb') as f:
            f.write(response.content)
        print(f"Downloaded PDF: {output_filename}")
    except requests.exceptions.HTTPError as e:
        print(f"HTTP error: {e}")
    except requests.exceptions.RequestException as e:
        print(f"Error downloading PDF: {e}")

# Main function to download the book PDF
def main():
    book_name = input("Enter the book name: ")
    author_name = input("Enter the author name: ")

    # Search for the book
    book_url = search_books(book_name, author_name)
    
    if book_url:
        # Download the PDF
        output_filename = f"{book_name.replace(' ', '_')}.pdf"
        download_pdf(book_url, output_filename)
    else:
        print("No PDF available for this book.")

if __name__ == "__main__":
    main()

Enter the book name:  from ml algorithm to genai & llms
Enter the author name:  Aman kharwal


Result 1:
Title: From ML Algorithms to GenAI & LLMs
Author(s): Aman Kharwal
Book available in PDF format: http://books.google.co.in/books/download/From_ML_Algorithms_to_GenAI_LLMs-sample-pdf.acsm?id=LCAqEQAAQBAJ&format=pdf&output=acs4_fulfillment_token&dl_type=sample&source=gbs_api
Downloaded PDF: from_ml_algorithm_to_genai_&_llms.pdf


In [62]:
import requests
import os

# Function to search for a book by ISBN using Open Library API
def search_by_isbn(isbn):
    # URL for Open Library API (searching by ISBN)
    url = f"https://openlibrary.org/api/books?bibkeys=ISBN:{isbn}&format=json&jscmd=data"
    
    # Send a GET request to Open Library API
    response = requests.get(url)
    
    if response.status_code == 200:
        book_data = response.json()
        
        # Check if book data is available
        key = f"ISBN:{isbn}"
        if key in book_data:
            book = book_data[key]
            title = book.get('title', 'No title')
            authors = book.get('authors', ['No authors listed'])
            print(f"Book Title: {title}")
            print(f"Author(s): {', '.join([author['name'] for author in authors])}")
            
            # Check if book has a PDF link
            if 'pdf' in book:
                pdf_url = book['pdf'].get('url', '')
                if pdf_url:
                    print(f"PDF available: {pdf_url}")
                    return pdf_url
                else:
                    print("No PDF link available.")
            else:
                print("No PDF link available.")
        else:
            print("Book not found by ISBN.")
    else:
        print(f"Error: Unable to fetch data from Open Library API for ISBN: {isbn}")
    return None

# Function to download the PDF from the URL
def download_pdf(pdf_url, output_filename):
    try:
        # Send a GET request to download the PDF
        response = requests.get(pdf_url)
        response.raise_for_status()  # Raise an error for bad responses
        
        # Create the folder to store PDF if it doesn't exist
        os.makedirs("downloaded_books", exist_ok=True)
        
        # Save the content as a PDF file
        with open(os.path.join("downloaded_books", output_filename), 'wb') as f:
            f.write(response.content)
        print(f"Downloaded PDF: {output_filename} (size: {len(response.content)/1024/1024:.2f} MB)")
    except requests.exceptions.HTTPError as e:
        print(f"HTTP error: {e}")
    except requests.exceptions.RequestException as e:
        print(f"Error downloading PDF: {e}")

# Main function to download the book PDF using ISBN
def main():
    isbn = input("Enter the ISBN of the book: ")

    # Search for the book by ISBN
    pdf_url = search_by_isbn(isbn)
    
    if pdf_url:
        # Sanitize the output filename to avoid special character issues
        sanitized_filename = f"book_{isbn}.pdf"
        
        # Download the PDF
        download_pdf(pdf_url, sanitized_filename)
    else:
        print("No PDF available for this book.")

if __name__ == "__main__":
    main()

Enter the ISBN of the book:  9780140328721


Book Title: Fantastic Mr. Fox
Author(s): Roald Dahl
No PDF link available.
No PDF available for this book.
